In [1]:
import sys
!{sys.executable} -m pip install mysql-connector sqlalchemy
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from config import *

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
disaster_frame = pd.read_csv('emdata.csv',names=['DisDate','Iso','DisType','Deaths','DollarDamage'])
disaster_frame.head()

,DisDate,Iso,DisType,Deaths,DollarDamage
0,10/06/1954,AFG,Earthquake,2000,0
1,10/06/1956,AFG,Earthquake,100,25000
2,/07/1956,AFG,Flood,51,0
3,/04/1963,AFG,Flood,107,0
4,//1969,AFG,Drought,0,200


In [3]:
def split_dates(frame):
    dates = frame.DisDate
    split_dates = dates.apply(lambda x: x.split('/'))
    years_only = split_dates.apply(lambda x: not x[0] and not x[1])
    months_only = split_dates.apply(lambda x: not x[0] and not not x[1])
    full_dates = split_dates.apply(lambda x: all(x))
    return frame[years_only], frame[months_only], frame[full_dates]

In [4]:
def is_date(row,year=None,month=None,day=None):
    retVal = True
    if year:
        retVal = retVal and row['DisDate'].split('/')[2] == str(year)
    if month:
        retVal = retVal and row['DisDate'].split('/')[1] == str(month)
    if day:
        retVal = retVal and row['DisDate'].split('/')[0] == str(day)
    return retVal

# Prunes the columns from the comtrade API

In [5]:
def prune_columns(frame):
  important_columns = ['yr','rgCode', 'rtCode','rtTitle', 'ptCode', 'ptTitle', 'TradeQuantity', 'cmdCode', 'TradeValue']
  frame = frame[important_columns]
  frame['cmdCode'] = pd.to_numeric(frame['cmdCode'], errors='coerce', downcast='integer')
  frame.loc[frame['cmdCode'].isna(),'cmdCode'] = -1
  frame.cmdCode = frame.cmdCode.astype(int)
  return frame

In [6]:
def eng_connector(user=None,password=None,host=None,port=None,database=None):
    engine = create_engine('mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'.format(user=username,password=password,host=hostname,port=port,database=database), echo=False)
    cnx = engine.connect()
    return engine, cnx

In [7]:
engine, con = eng_connector(user=username,password=password,host=hostname,port=port,database=database)

In [8]:
def get_trade_data(connector, query="select * from cocoapuffs"):
    all_data = pd.read_sql(query,connector)
    #Converts the cmdCode series to the integer datatype
    all_data.cmdCode = pd.to_numeric(all_data.cmdCode,downcast='integer',errors='coerce')
    #Assert all of the cmdCodes are not null
    assert(all(all_data.cmdCode.notnull()))
    return all_data

In [9]:
trades = get_trade_data(con)
trades

,yr,rgCode,rtCode,rtTitle,ptCode,ptTitle,TradeQuantity,cmdCode,TradeValue
0,2016,1,276,Germany,56,Belgium,45234634.0,1701,30278272
1,2016,2,276,Germany,56,Belgium,47845776.0,1701,29128813
2,2016,1,276,Germany,56,Belgium,34906463.0,1801,119490227
3,2016,2,276,Germany,56,Belgium,445212.0,1801,1449538
4,2016,1,276,Germany,56,Belgium,849687.0,1804,7293059
5,2016,2,276,Germany,56,Belgium,16735401.0,1804,109612364
6,2016,1,276,Germany,251,France,4038964.0,1804,25895475
7,2016,2,276,Germany,251,France,5237039.0,1804,34312432
8,2016,1,276,Germany,251,France,978112.0,1801,2424948
9,2016,2,276,Germany,251,France,1007992.0,1801,3403256


In [10]:
#cmdCode -1 is the total value of that commodity to the world
#rtCode 276 is Germany
trades[(trades['cmdCode']==-1) & (trades['rtCode']!=276)]

,yr,rgCode,rtCode,rtTitle,ptCode,ptTitle,TradeQuantity,cmdCode,TradeValue
660,2017,2,8,Albania,0,World,0.0,-1,2261555720
661,2017,2,12,Algeria,0,World,0.0,-1,35191117416
662,2017,2,51,Armenia,0,World,0.0,-1,2041341086
663,2017,2,56,Belgium,0,World,0.0,-1,429980167539
664,2017,2,70,Bosnia Herzegovina,0,World,0.0,-1,6366899729
665,2017,2,76,Brazil,0,World,0.0,-1,217739177077
666,2017,2,100,Bulgaria,0,World,0.0,-1,30181926138
667,2017,2,124,Canada,0,World,0.0,-1,420501790917
668,2017,2,144,Sri Lanka,0,World,0.0,-1,11741036786
669,2017,2,152,Chile,0,World,0.0,-1,68305476034
